In [256]:
!pip install yfinance
!pip install pytickersymbols

In [257]:
import yfinance
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from pytickersymbols import PyTickerSymbols
from google.colab import files
from datetime import datetime, timedelta

In [258]:
start="2001-01-01"
end="2021-01-01"
period = 20
multiplier = 2

In [259]:
stock_data = PyTickerSymbols()

nasdaq_100=pd.DataFrame(stock_data.get_stocks_by_index('NASDAQ 100'))
nasdaq_100 = nasdaq_100[nasdaq_100.symbol != 'CTRP'].reset_index().symbol.to_list()

sp_500=pd.DataFrame(stock_data.get_stocks_by_index('S&P 500'))
sp_500 = sp_500[sp_500.symbol != 'SSE']
sp_500 = sp_500[sp_500.symbol != 'RTN']
sp_500 = sp_500[sp_500.symbol != 'CXO']
sp_500 = sp_500[sp_500.symbol != 'JEC']
sp_500 = sp_500[sp_500.symbol != 'TIF'].reset_index().symbol.to_list()

total_tickers = nasdaq_100 + sp_500

today = datetime.today().strftime('%Y-%m-%d')

In [260]:
len(total_tickers)

592

In [261]:
#FILTRO PARA TICKERS
# tickers_list = []
# for i in range(len(total_tickers)):
#   ticker = yfinance.download(total_tickers[i], today)[['Close', 'Volume']]
#   if ticker['Close'][0] > 30 and ticker['Close'][0] < 120 and ticker['Volume'][0] > 5000000:
#     tickers_list.append(total_tickers[i])

In [262]:
#tickers_list

In [263]:
#COPIADOS DEL FILTRO EL 27/01/2021
tickers_list = ['CSCO','INTC','WBA','ATVI','AMAT','CMCSA','CSX','EBAY','FISV','GILD','JD','KHC','MU','MDLZ','SBUX','EXC','AMD','FOXA','KDP',
 'MRVL','AXP','CVX','CSCO','KO','DOW','XOM','INTC','MRK','PFE','VZ','WBA','ATVI','AMAT','CMCSA','CSX','DISCA','DISH','EBAY','FISV','GILD','KHC',
 'MU','MDLZ','STX','SBUX','WDC','ABT','ABBV','MO','AIG','BK','BMY','COF','C','CL','COP','CVS','EXC','GM','MDT','MET','MS','NEE','ORCL','SPG','USB',
 'WFC','AMD','BWA','BSX','CNC','CFG','ED','GLW','DAL','EOG','FE','FOXA','GIS','IFF','IRM','K','KR','LB','MPC','NEM','SCHW','LUV','SYF','TPR','TJX',
 'TWTR','UAL','VFC','CARR','DOW','LVS','VIAC']

len(tickers_list)

98

In [264]:
tickers_list = list(set(total_tickers))
len(tickers_list)

514

In [265]:
#get stock info
#yfinance.Ticker("msft").info

In [266]:
# ticker_list = nasdaq_100.symbol

# for i in range(len(ticker_list)):
#   if yfinance.Ticker(ticker_list[i]).info['averageDailyVolume10Day'] > 2000000 and yfinance.Ticker(ticker_list[i]).info['open'] > 30 and yfinance.Ticker(ticker_list[i]).info['open'] < 120:
#     print(ticker_list[i])

In [267]:
df_totales = pd.DataFrame(['Total Trades', 'Green Trades', 'Red Trades', 'Green PL (Rs)', 'Red PL (Rs)', 'PL TOTAL'], 
                          index=['Total Trades', 'Green Trades', 'Red Trades', 'Green PL (Rs)', 'Red PL (Rs)', 'PL TOTAL']).drop(columns=0)

In [268]:
#tickers_list = ['MLM'] #PARA PRUEBA CON UN SOLO TICKER

for j in range(len(tickers_list)):

    ticker = yfinance.Ticker(tickers_list[j])
    df = ticker.history(interval="1d",start=start,end=end)
    df['UpperBand'] = df['Close'].rolling(period).mean() + df['Close'].rolling(period).std() * multiplier
    df['LowerBand'] = df['Close'].rolling(period).mean() - df['Close'].rolling(period).std() * multiplier
    #df['AVG Volume'] =
    df['AVG Daily Range'] = df['High'].subtract(df['Low']).rolling(14).mean()

    df2 = df[['Open', 'High', 'Low', 'Close', 'AVG Daily Range', 'UpperBand', 'LowerBand']]
    df2 = df2.dropna()

    df2['Trigger'] = np.nan
    #SE VUELVEN A REVISAR LOS FILTROS DE PRECIO Y RANGO DIARIO
    
    for i in range(2, len(df2)):
      if (df2['Close'][i-2] > 30 and df2['Close'][i-2] < 120 and df2['AVG Daily Range'][i-2] >= 1.5 and df2['Close'][i-2] < df2['Open'][i-2] 
      and ((df2['High'][i-2] + df2['Low'][i-2]) / 2) < df2['LowerBand'][i-2]): #and df2['Close'][i-2] < df2['Close'][i-1]):
        stop = df2['Low'][i-2] # STOP ES EL LOW DE LOS DOS CANDLES PREVIOS 
        if df2['Open'][i] >= stop:
          df2['Trigger'][i] = True
        else:
          df2['Trigger'][i] = False
    
    # df2['Trigger'] = np.nan
    # for i in range(1, len(df2)):
    #   stop = (df2['High'][i-1] + df2['Low'][i-1]) / 2 #PARA TRIGGEAR: UN GAP MINIMO DE 50 CENTAVOS (OSEA DEL CLOSE ANTERIOR)
    #   if df2['Outside'][i-1] == True and df2['Open'][i] >= (df2['Close'][i-1] + 0.5) and (df2['Open'][i] - 0.5) >= stop: #(df2['Close'][i-1] + 0.5)
    #     df2['Trigger'][i] = 1.0
    
    df2['Green'] = np.nan
    df2['Red'] = np.nan

    for i in range(1, len(df2)):
      if df2['Trigger'][i] == 1.0:
        stop = df2['Low'][i-2] # STOP ES EL LOW DEL DOS CANDLES PREVIOS 
        if df2['Low'][i] < stop:
          df2['Red'][i] = -1
        elif (df2['Close'][i] - df2['Open'][i])/(df2['Open'][i]-stop) > 0: 
          df2['Green'][i] = (df2['Close'][i] - df2['Open'][i])/(df2['Open'][i]-stop)
        else:
          df2['Red'][i] = (df2['Close'][i] - df2['Open'][i])/(df2['Open'][i]-stop)

    df_totales.insert(j, tickers_list[j], [df2['Trigger'].count(), df2['Green'].count(), df2['Red'].count(),
                                           df2['Green'].sum(),df2['Red'].sum(), (df2['Green'].sum() + df2['Red'].sum())])

In [269]:
df_totales['sum'] = df_totales.sum(axis=1)
df_totales

,LDOS,MLM,HAS,BR,CDW,FMC,NWSA,NRG,STX,TSLA,NKE,JBHT,ROK,REG,LVS,CTXS,IVZ,HII,YUM,FLIR,AMD,GOOGL,ADM,AMAT,LNC,MCK,TFC,SLG,PBCT,JNPR,PG,VRSK,F,JD,VRSN,CPRT,BMY,ZBRA,ABMD,IFF,...,RE,APD,HLT,MNST,PLD,EW,IEX,ITW,LW,MTCH,CHKP,URI,CMI,DLTR,SGEN,FLS,RSG,AZO,CMG,FFIV,VIAC,NLOK,KMB,LUV,XLNX,WEC,SWKS,KLAC,JCI,UHS,NWL,PDD,LHX,V,ACN,EVRG,LYV,JNJ,DTE,sum
Total Trades,8.000000,17.000000,8.000000,10.000000,5.000000,6.000000,0.0,0.0,2.000000,19.000000,7.000000,11.000000,16.000000,4.00000,6.000000,14.000000,0.0,8.000000,4.000000,3.000000,2.00000,1.000000,0.0,5.000000,5.000000,8.000000,1.0,30.000000,0.0,1.000000,5.000000,6.000000,0.0,2.000000,3.00000,2.000000,1.000000,15.000000,2.000000,12.000000,...,6.000000,13.000000,3.000000,5.000000,7.000000,6.000000,1.000000,2.000000,7.000000,4.000000,12.000000,23.000000,26.000000,15.000000,15.000000,1.000000,2.0,10.000000,9.000000,28.000000,0.0,0.0,10.000000,2.000000,6.000000,4.000000,18.000000,4.000000,1.000000,17.000000,0.0,1.000000,1.000000,3.0000,8.000000,4.000000,6.000000,5.000000,5.000000,3582.000000
Green Trades,4.000000,7.000000,3.000000,5.000000,3.000000,1.000000,0.0,0.0,1.000000,6.000000,2.000000,7.000000,8.000000,0.00000,1.000000,7.000000,0.0,5.000000,0.000000,1.000000,0.00000,0.000000,0.0,0.000000,0.000000,2.000000,0.0,14.000000,0.0,0.000000,4.000000,3.000000,0.0,2.000000,1.00000,0.000000,0.000000,6.000000,1.000000,7.000000,...,2.000000,5.000000,3.000000,0.000000,1.000000,3.000000,1.000000,2.000000,2.000000,2.000000,6.000000,11.000000,15.000000,4.000000,5.000000,1.000000,0.0,6.000000,5.000000,12.000000,0.0,0.0,9.000000,1.000000,2.000000,2.000000,6.000000,2.000000,0.000000,7.000000,0.0,1.000000,1.000000,2.0000,5.000000,3.000000,4.000000,2.000000,1.000000,1596.000000
Red Trades,2.000000,8.000000,5.000000,5.000000,2.000000,5.000000,0.0,0.0,1.000000,12.000000,4.000000,4.000000,8.000000,3.00000,5.000000,7.000000,0.0,2.000000,4.000000,2.000000,1.00000,1.000000,0.0,4.000000,5.000000,5.000000,1.0,14.000000,0.0,1.000000,1.000000,3.000000,0.0,0.000000,2.00000,2.000000,1.000000,9.000000,1.000000,5.000000,...,4.000000,8.000000,0.000000,4.000000,6.000000,3.000000,0.000000,0.000000,3.000000,2.000000,6.000000,10.000000,10.000000,10.000000,10.000000,0.000000,1.0,4.000000,4.000000,13.000000,0.0,0.0,1.000000,1.000000,4.000000,2.000000,12.000000,2.000000,1.000000,9.000000,0.0,0.000000,0.000000,1.0000,3.000000,0.000000,2.000000,3.000000,2.000000,1749.000000
Green PL (Rs),1.224239,8.370096,2.304811,4.085367,3.083715,0.118646,0.0,0.0,0.328829,6.376363,0.411322,4.030843,4.202193,0.00000,0.261785,7.452754,0.0,1.742105,0.000000,0.089687,0.00000,0.000000,0.0,0.000000,0.000000,0.873035,0.0,5.990711,0.0,0.000000,10.110986,3.211888,0.0,0.659483,0.34615,0.000000,0.000000,1.662460,0.891889,3.680602,...,0.713547,1.589439,2.105718,0.000000,0.098830,0.971605,1.251612,0.390708,0.723696,1.596251,2.634378,6.490084,8.280912,0.556505,1.387629,0.149589,0.0,3.985447,1.496568,13.417710,0.0,0.0,30.029195,0.276041,0.803166,0.557638,3.185586,1.609634,0.000000,5.846013,0.0,0.402667,0.202165,0.8536,2.948579,1.626656,1.648276,0.071309,1.706315,1094.754158
Red PL (Rs),-0.184503,-5.176517,-2.387650,-2.193808,-1.168480,-1.918318,0.0,0.0,-1.000000,-8.216000,-2.505594,-2.251432,-4.182623,-2.35909,-3.278413,-4.980325,0.0,-1.453071,-1.756459,-0.996812,-0.03546,-0.054531,0.0,-3.721649,-2.572212,-2.568618,-1.0,-8.040930,0.0,-0.439918,-0.503435,-1.767551,0.0,0.000000,-2.00000,-1.741938,-0.082802,-5.771092,-1.000000,-1.959500,...,-1.325358,-4.613335,0.000000,-2.643184,-5.306121,-1.511136,0.000000,0.000000,-2.175168,-1.793603,-2.522757,-5.908543,-6.694188,-5.263779,-5.190983,0.000000,-1.0,-2.406089,-2.334833,-7.677979,0.0,0.0,-0.514283,-0.107144,-2.525335,-0.296404,-7.769731,-2.000000,-0.526668,-6.278820,0.0,0.000000,0.000000,-1.0000,-0.799256,0.000000,-2.000000,-0.874090,-1.563468,-1004.597603
PL TOTAL,1.039736,3.193579,-0.082839,1.891558,1.915235,-1.799672,0.0,0.0,-0.671171,-1.839636,-2.094272,1.779411,0.019569,-2.35909,-3.016628,2.472429,0.0,0.289034

In [270]:
df2[(df2['Trigger']==1.0)]
#df2[(df2['Trigger']==1.0) | (df2['Outside']==1.0)].head(60)
#df2.loc['2015-04-29':'2015-05-01']

,Open,High,Low,Close,AVG Daily Range,UpperBand,LowerBand,Trigger,Green,Red
Date,,,,,,,,,,
2015-02-18,65.929764,67.890144,65.785146,67.890144,1.677455,76.059609,65.239478,1.0,1.706315,NaN
2019-11-12,115.221440,115.488708,114.620089,115.087807,1.862018,126.335048,114.274567,1.0,NaN,-1.000000
2020-03-03,112.265919,114.989320,107.839176,108.763016,3.176390,139.682822,109.258614,1.0,NaN,-0.563468
